In [1]:
using Plots
using Flux, Flux.Data.MNIST ## this is the Julia package for deep learning 
using Flux: onehotbatch, argmax, crossentropy, throttle, mse
using Base.Iterators: repeated, partition

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [2]:
include("postgresql.jl")
include("TunnelingModule.jl")

loaded


knn (generic function with 2 methods)

In [3]:
connection_dir = "./pitch-tunneling"

user, pw = open("pw_pitch_tunneling.txt") do file
    readlines(file)
end

username = user
password = pw

conn = connect_pitch_tunneling(connection_dir, username, password)

Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.


PostgreSQL connection (CONNECTION_OK) with parameters:
  user = user1
  password = ********************
  dbname = pitch-tunneling
  host = 141.211.55.211
  port = 58421
  client_encoding = UTF8
  application_name = LibPQ.jl
  sslmode = verify-ca
  sslcompression = 1
  sslcert = ./pitch-tunneling/postgresql.crt
  sslkey = ./pitch-tunneling/postgresql.key
  sslrootcert = ./pitch-tunneling/root.crt
  krbsrvname = postgres
  target_session_attrs = any

In [19]:
query = """
  SELECT ordinal_position,
         column_name,
         data_type,
         numeric_precision
    FROM information_schema.columns
   WHERE table_name = 'pitchtunnelingtrain'
ORDER BY -ordinal_position;
"""
result = execute(conn, query)

#display list of columns as a data frame
data = Data.stream!(result, Data.Table)
DataFrame(data)

,ordinal_position,column_name,data_type,numeric_precision
,Int32⍰,String⍰,String⍰,Int32⍰
1,48,runvalue,character varying,missing
2,47,pitchnumber,integer,32
3,46,atbatnumber,integer,32
4,45,trajectorylocationz,double precision,53
5,44,trajectorylocationy,double precision,53
6,43,trajectorylocationx,double precision,53
7,42,trajectoryverticalbreak,double precision,53
8,41,trajectoryhorizontalapproachangle,double precision,53
9,40,trajectoryverticalapproachangle,double precision,53


In [4]:
#lets look at a suggested subset of the data: left handed pichers, left handed batters, and only fastballs/sinkers
#leave out missing runvalue data
query = """
    SELECT releasepositionx, releasepositionz, tunnellocationx, tunnellocationz, trajectorylocationx, trajectorylocationz, runvalue
    FROM pitchtunnelingtrain
    WHERE islhp = '1' AND islhb = '1' AND (pitchtype = 'FF' OR pitchtype = 'FT' OR pitchtype = 'SI') AND runvalue IS NOT NULL;
"""

datatrain = stream_to_dataframe(conn, query)

query = """
    SELECT releasepositionx, releasepositionz, tunnellocationx, tunnellocationz, trajectorylocationx, trajectorylocationz, runvalue
    FROM pitchtunnelingtest
    WHERE islhp = '1' AND islhb = '1' AND (pitchtype = 'FF' OR pitchtype = 'FT' OR pitchtype = 'SI') AND runvalue IS NOT NULL AND runvalue != 'NA';
"""

datatest = stream_to_dataframe(conn, query)

,releasepositionx,releasepositionz,tunnellocationx,tunnellocationz,trajectorylocationx,trajectorylocationz,runvalue
,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,String⍰
1,0.85044,6.22609,0.663672,4.39568,1.08375,1.55677,0.0470650797301319
2,0.311899,6.15658,-0.909834,4.77586,-1.23874,2.73032,0.0470650797301319
3,0.519096,6.40694,0.218801,5.45135,0.429317,3.58656,0.0470650797301319
4,0.877857,6.19029,-0.0691493,4.83067,-0.139442,2.65096,-0.0655810812636067
5,0.866311,6.05188,-0.181407,4.6771,-0.118078,2.44635,-0.0655810812636067
6,0.413331,6.32047,-0.29682,4.83414,-0.324301,2.65404,-0.309049625851062
7,0.563192,6.11328,-0.471595,4.70428,-0.728872,2.32478,-0.0599418042736001
8,0.679952,6.03171,0.0652037,6.09701,0.213531,4.87167,0.0471762196274407
9,0.851174,6.28286,0.418291,5.29125,0.777429,3.29639,-0.296284041204902


In [5]:
Xtrain = convert(Array,datatrain[1:6]); #only need column index
Ytrain = parse.(Float64,datatrain[7]);

Xtest = convert(Array,datatest[1:6]);
Ytest = parse.(Float64,datatest[7]);

In [7]:
lin_reg_mse(Xtrain,Ytrain,Xtest,Ytest)

([-0.00540383, 0.00724621, -0.00622481], 0.04226133219210863)

In [25]:
result = nn_full(Xtrain,Ytrain,Xtest,Ytest)

[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.507253 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.402005 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.313142 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.2399 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.181155 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.135547 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.101639 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0778821 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.062591 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0539187 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0499479 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0488635 (tracked)]
[loss(Xtrain', ytrain')] =

[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0416129 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0416116 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0416103 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.041609 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0416078 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0416065 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0416053 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0416041 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0416028 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0416016 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0416004 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0415992 (tracked)]
[loss(Xtrain', yt

(Any[Flux.Tracker.TrackedReal{Float64}[-0.447435 (tracked) -0.328902 (tracked) … 0.244961 (tracked) 0.104613 (tracked); -0.51236 (tracked) -0.28315 (tracked) … -0.408038 (tracked) 0.214888 (tracked); … ; 0.367405 (tracked) 0.39632 (tracked) … 0.416876 (tracked) -0.265377 (tracked); 0.047679 (tracked) -0.150402 (tracked) … -0.0905258 (tracked) -0.626155 (tracked)], Flux.Tracker.TrackedReal{Float64}[-0.0352616 (tracked), 0.0360583 (tracked), -0.0362748 (tracked), 0.0278535 (tracked), 0.0284771 (tracked), 0.0391921 (tracked), 0.0315743 (tracked), -0.0296736 (tracked)], Flux.Tracker.TrackedReal{Float64}[-0.185332 (tracked) 0.256562 (tracked) … 0.151978 (tracked) -0.373252 (tracked); -0.671203 (tracked) 0.0446241 (tracked) … -0.111253 (tracked) 0.238903 (tracked); … ; -0.323107 (tracked) 0.313596 (tracked) … -0.172884 (tracked) 0.134076 (tracked); -0.0526991 (tracked) -0.459154 (tracked) … 0.452175 (tracked) -0.493301 (tracked)], Flux.Tracker.TrackedReal{Float64}[0.0302378 (tracked), 0.0022

In [10]:
result[3]' ./ Ytest

Tracked 13072×1 Array{Float64,2}:
   -0.35338692575043806
   -1.103637985752589  
   -0.8804163021974123 
   -0.27582087811212025
   -0.48300280575841814
   -0.48597062974732   
   -0.700303990026942  
   -0.9956919619242015 
   -0.9188194931995148 
   -0.8744544804001103 
   -0.8845756996157225 
    0.175583391006159  
   -1.3464794609258646 
    ⋮                  
   -1.2143813531018435 
   -0.9524319808105112 
 -Inf                  
 -Inf                  
   -0.4812543123406995 
   -0.861099428129825  
   -0.84555220574652   
   -0.7455662558049605 
   -0.5943573672785717 
   -0.47745114626075164
   -0.9089687160471384 
   -0.40093847041334546

In [7]:
include("TunnelingModule.jl")
ratio_result = nn_full(calc_ratios(Xtrain),Ytrain,calc_ratios(Xtest),Ytest)
#we see rse improvement training nn with ratios instead of direct features, and improvement over linear regression

[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.203025 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.163595 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.130989 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.10511 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0855203 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0714741 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0619917 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0559716 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0524272 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0504833 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0494217 (tracked)]
[loss(Xtrain', ytrain')] = Flux.Tracker.TrackedReal{Float64}[0.0487211 (tracked)]
[loss(Xtrain', ytrain

(Any[Flux.Tracker.TrackedReal{Float64}[0.339787 (tracked) 0.360122 (tracked); -0.531478 (tracked) 0.148897 (tracked); … ; 0.523311 (tracked) -0.312323 (tracked); 0.112668 (tracked) -0.464883 (tracked)], Flux.Tracker.TrackedReal{Float64}[0.0352244 (tracked), 0.0257914 (tracked), -0.00266105 (tracked), -0.0320527 (tracked), -0.031776 (tracked), -0.0326534 (tracked), 0.0598904 (tracked), -0.0319599 (tracked)], Flux.Tracker.TrackedReal{Float64}[-0.302207 (tracked) -0.476652 (tracked) … 0.164751 (tracked) -0.318765 (tracked); -0.187486 (tracked) -0.238095 (tracked) … -0.0844026 (tracked) 0.111024 (tracked); … ; 0.119912 (tracked) -0.193291 (tracked) … 0.231696 (tracked) 0.333446 (tracked); -0.204269 (tracked) 0.163341 (tracked) … -0.262132 (tracked) -0.0296093 (tracked)], Flux.Tracker.TrackedReal{Float64}[-0.0305733 (tracked), -0.0360017 (tracked), -0.0367945 (tracked), 0.0327738 (tracked), 0.0411822 (tracked), -0.19886 (tracked), 0.0131314 (tracked), 0.0621807 (tracked), 0.0480383 (tracked

In [7]:
#use both cores for parallel computation of k nearest neighbors
addprocs(2)
workers()

2-element Array{Int64,1}:
 2
 3

In [6]:
kmax = 20
mses = zeros(kmax)
for i = 1:kmax
    mses[i] = knn(calc_ratios(Xtrain),Ytrain,calc_ratios(Xtest),Ytest,i)[2]
end

In [7]:
mses

20-element Array{Float64,1}:
 0.08412426537645191 
 0.07544569638769537 
 0.07034948004071484 
 0.06575460251240135 
 0.06011428150189304 
 0.060289366441266354
 0.06138133230033914 
 0.06516246349648912 
 0.06368075971044358 
 0.06227763902810502 
 0.06126535737033763 
 0.06313362176557585 
 0.0626315779960878  
 0.061810895898097164
 0.060108245907869715
 0.05998473053167588 
 0.06137934629223049 
 0.061396282533498735
 0.060679456734218214
 0.061492136576082504